## Training On Cloud Space

This code clone the 3dgs repository and necessary resources on temporary storage space and produce trained model

In [ ]:
%cd /content
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile

%cd /content/gaussian-splatting
!pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/gaussian-splatting/submodules/simple-knn

!wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
!unzip tandt_db.zip

!python train.py -s /content/gaussian-splatting/tandt/train

## Training On Local Drive

Instead, change the file papth to store the model on local google drive

In [ ]:
%cd /content/drive/MyDrive/3dgs
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile

%cd /content/drive/MyDrive/3dgs/gaussian-splatting
!pip install -q /content/drive/MyDrive/3dgs/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/drive/MyDrive/3dgs/gaussian-splatting/submodules/simple-knn

!wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
!unzip tandt_db.zip

!python train.py -s /content/drive/MyDrive/3dgs/gaussian-splatting/tandt/train

# !wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/GaussianViewTest.zip
# !unzip GaussianViewTest.zip
# !python render.py -m /content/gaussian-splatting/GaussianViewTest/model
# !ffmpeg -framerate 3 -i /content/gaussian-splatting/GaussianViewTest/model/train/ours_30000/renders/%05d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -r 3 -pix_fmt yuv420p /content/renders.mp4
# !ffmpeg -framerate 3 -i /content/gaussian-splatting/GaussianViewTest/model/train/ours_30000/gt/%05d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -r 3 -pix_fmt yuv420p /content/gt.mp4 -y

## Collect Gassian Splat Dataset

### Set up environment

Assume the repository is cloned in local drive, install the modified module `diff-gaussian-rasterization` for 
- Two cells below have same use, but second will produce detailed error output from compilation
- Our rasterization module will record the gaussian splats that contribute to the pixel; that is, we excluded the splats that are too far away from the pixel and are occluded entirely by the splats with smaller depth. The second situation is rare in our statistic check.

In [ ]:
%cd /content/drive/MyDrive/3dgs/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -e .

In [ ]:
%cd /content/drive/MyDrive/3dgs/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -e .

Install the `simple-knn` and `plyfile` module

In [ ]:
%cd /content/drive/MyDrive/3dgs/gaussian-splatting/submodules/simple-knn
!pip install -e .
!pip install -q plyfile

## Generate dataset of gaussian
Format: <`pixelNum`, `out_color_r`,`out_color_g`, `out_color_b`, `bg_r`, `bg_g`, `bg_b`, `alpha_1`, `depth_1`, `r_1`, `g_1`, `b_1`,....>

### Flags
`-m`: path for the model 

`-s`: path for source dataset

`--view_index`: index of camera position

In [ ]:
%cd /content/drive/MyDrive/3dgs/gaussian-splatting
!python render_ray.py -m /content/drive/MyDrive/3dgs/data/pre-trained/3DGS/playroom -s /content/drive/MyDrive/3dgs/data/tandt/playroom --view_index 20 --skip_test

Get smaller dataset for faster sanity check.

`n_rows` determine the size of smaller dataset.

In [ ]:
import pandas as pd

# File paths
input_file_path = '/content/drive/MyDrive/3dgs/gaussian-splatting/alpha_vals.csv'
output_file_path = '/content/drive/MyDrive/3dgs/gaussian-splatting/update_playroom_20.csv'

n_rows = 10000

# Read part of the CSV
df = pd.read_csv(input_file_path, nrows=n_rows)
df.to_csv(output_file_path, index=False)
